In [29]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import pandas as pd
import numpy as np

import inventario_finanzas_int.funciones_auxiliares as fa

pd.set_option("display.max_columns", None)

In [31]:
df_mobiliario = pd.read_csv("../data/processed/df_procesada_mobiliarios.csv")
df_equipos_medicos = pd.read_csv("../data/processed/df_procesada_equipos_medicos.csv")

# lee Industriales, y elimina las columnas completamente vacias
df_industriales = pd.read_csv("../data/processed/df_procesada_industriales.csv")
df_industriales = df_industriales.dropna(axis=1, how="all")

# Lee Informaticos
df_informaticos = pd.read_csv("../data/processed/df_procesada_informaticos.csv")

In [32]:
union_bienes_con_informatico = pd.concat(
    [df_mobiliario, df_equipos_medicos, df_industriales, df_informaticos]
)
union_bienes_con_informatico["conteo"] = 1

union_bienes_sin_informatico = pd.concat([df_mobiliario, df_equipos_medicos, df_industriales])
union_bienes_sin_informatico["conteo"] = 1

## Asignacion de Correlativos

In [35]:
# Solamente obtiene los bienes del hospital
bienes_sin_funcionarios = union_bienes_sin_informatico.query("propiedad == 'INT'").copy()

# Asigna numero correlativo
bienes_sin_funcionarios["correlativo_nuevo"] = [
    f"2025-{i}" for i in range(1, len(bienes_sin_funcionarios) + 1)
]

In [36]:
bienes_sin_funcionarios.to_excel("../data/interim/matriz_bienes_INT.xlsx", index=False)

In [38]:
conteo_equipos_con_informatico = pd.pivot_table(
    union_bienes_con_informatico,
    columns="tipo_bien",
    index="unidadservicio_clinico",
    values="conteo",
    fill_value=0,
    aggfunc="sum",
)

conteo_equipos_sin_informatico = pd.pivot_table(
    union_bienes_sin_informatico,
    columns="tipo_bien",
    index="unidadservicio_clinico",
    values="conteo",
    fill_value=0,
    aggfunc="sum",
)